In [59]:
import cv2 as cv
import numpy as np
import math
import time
from matplotlib import pyplot as plt
import os

n = 2  # número de janelas em colunas
m = 2  # número de janelas em linhas
EPSILON = 0.00001
GAMMA = 1  # Controle de brilho
IDEAL_VARIANCE = 0.35  # Controle de contraste

IMAGE_PATH = '132623882_2940025776268895_9131617843404056123_n.png'  # Coloque o caminho correto da imagem

# Verifique se a imagem existe
if not os.path.exists(IMAGE_PATH):
    print(f'Imagem não encontrada: {IMAGE_PATH}')
else:
    img = cv.imread(IMAGE_PATH)
    layer = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    WIDTH = layer.shape[1]
    HEIGHT = layer.shape[0]
    x0, x1, y0, y1 = 0, WIDTH - 1, 0, HEIGHT - 1

    def phy(value):
        return 0.5 * np.log((1 + value) / ((1 - value) + EPSILON))

    def multiplication(value1, value2):
        return phy(value1) * phy(value2)

    def norm(value):
        return abs(phy(value))

    def scalar_multiplication(scalar, value):
        s = (1 + value) ** scalar
        z = (1 - value) ** scalar
        return (s - z) / (s + z + EPSILON)

    def addition(value1, value2):
        return (value1 + value2) / (1 + (value1 * value2) + EPSILON)

    def subtract(value1, value2):
        return (value1 - value2) / (1 - (value1 * value2) + EPSILON)

    def C(m, i):
        return math.factorial(m) / ((math.factorial(i) * math.factorial(m - i)) + EPSILON)

    def qx(i, x):
        if x == WIDTH - 1:
            return 0
        return C(m, i) * (np.power((x - x0) / (x1 - x), i) * np.power((x1 - x) / (x1 - x0), m))

    def qy(j, y):
        if y == HEIGHT - 1:
            return 0
        return C(n, j) * (np.power((y - y0) / (y1 - y), j) * np.power((y1 - y) / (y1 - y0), n))

    def p(i, j, x, y):
        return qx(i, x) * qy(j, y)

    def mapping(img, source, dest):
        return (dest[1] - dest[0]) * ((img - source[0]) / (source[1] - source[0])) + dest[0]

    e_layer_gray = mapping(layer, (0, 255), (-1, 1))

    def cal_ps_ws(m, n, w, h, gamma):
        ps = np.zeros((m, n, w, h))
        for i in range(m):
            for j in range(n):
                for k in range(w):
                    for l in range(h):
                        ps[i, j, k, l] = p(i, j, k, l)

        ws = np.zeros((m, n, w, h))
        for i in range(m):
            for j in range(n):
                ps_power_gamma = np.power(ps[i, j], gamma)
                for k in range(w):
                    for l in range(h):
                        ws[i, j, k, l] = ps_power_gamma[k, l] / (np.sum(ps[:, :, k, l]) + EPSILON)
        return ps, ws

    ps, ws = cal_ps_ws(m, n, WIDTH, HEIGHT, GAMMA)

    def cal_means_variances_lamdas(w, e_layer):
        means = np.zeros((m, n))
        variances = np.zeros((m, n))
        lamdas = np.zeros((m, n))

        def window_card(w):
            return np.sum(w)

        def window_mean(w, i, j):
            mean = 0
            for k in range(HEIGHT):
                for l in range(WIDTH):
                    mean = addition(mean, scalar_multiplication(w[i, j, l, k], e_layer[k, l]))
            return mean / window_card(w[i, j])

        def window_variance(w, i, j):
            variance = 0
            for k in range(HEIGHT):
                for l in range(WIDTH):
                    variance += w[i, j, l, k] * np.power(norm(subtract(e_layer[k, l], means[i, j])), 2)
            return variance / window_card(w[i, j])

        def window_lamda(w, i, j):
            return np.sqrt(IDEAL_VARIANCE) / (np.sqrt(variances[i, j]) + EPSILON)

        for i in range(m):
            for j in range(n):
                means[i, j] = window_mean(ws, i, j)
                variances[i, j] = window_variance(ws, i, j)
                lamdas[i, j] = window_lamda(ws, i, j)

        return means, variances, lamdas

    means, variances, lamdas = cal_means_variances_lamdas(ws, e_layer_gray)

    def window_enh(w, i, j, e_layer):
        return scalar_multiplication(lamdas[i, j], subtract(e_layer, means[i, j]))

    def image_enh(w, e_layer):
        new_image = np.zeros(e_layer.shape)
        width = e_layer.shape[1]
        height = e_layer.shape[0]
        for i in range(m):
            for j in range(n):
                win = window_enh(w, i, j, e_layer)
                w1 = w[i, j].T.copy()
                for k in range(width):
                    for l in range(height):
                        new_image[l, k] = addition(new_image[l, k], scalar_multiplication(w1[l, k], win[l, k]))
        return new_image

    def one_layer_enhancement(e_layer):
        new_E_image = image_enh(ws, e_layer)
        res_image = mapping(new_E_image, (-1, 1), (0, 255))
        res_image = np.round(res_image).astype(np.uint8)
        return res_image

    res_img = one_layer_enhancement(e_layer_gray)

    # Fuzzificação da imagem
    layer_b, layer_g, layer_r = cv.split(img)
    e_layer_b, e_layer_g, e_layer_r = mapping(layer_b, (0, 255), (-1, 1)), mapping(layer_g, (0, 255), (-1, 1)), mapping(layer_r, (0, 255), (-1, 1))
    e_layer_rgb = scalar_multiplication(1 / 3, addition(addition(e_layer_b, e_layer_g), e_layer_r))

    means, variances, lamdas = cal_means_variances_lamdas(ws, e_layer_rgb)

    plt.imsave('resultado.jpg', res_img)


In [60]:
import cv2 as cv
import numpy as np
import math
import os
from matplotlib import pyplot as plt

n = 2  # número de janelas em colunas
m = 2  # número de janelas em linhas
EPSILON = 0.00001
GAMMA = 1  # Controle de brilho
IDEAL_VARIANCE = 0.35  # Controle de contraste

IMAGE_PATH = '132623882_2940025776268895_9131617843404056123_n.png'  # Coloque o caminho correto da imagem

# Verifique se a imagem existe
if not os.path.exists(IMAGE_PATH):
    print(f'Imagem não encontrada: {IMAGE_PATH}')
else:
    img = cv.imread(IMAGE_PATH)
    HEIGHT, WIDTH, _ = img.shape  # Lida com imagens coloridas, com três canais
    x0, x1, y0, y1 = 0, WIDTH - 1, 0, HEIGHT - 1

    def phy(value):
        return 0.5 * np.log((1 + value) / ((1 - value) + EPSILON))

    def multiplication(value1, value2):
        return phy(value1) * phy(value2)

    def norm(value):
        return abs(phy(value))

    def scalar_multiplication(scalar, value):
        s = (1 + value) ** scalar
        z = (1 - value) ** scalar
        return (s - z) / (s + z + EPSILON)

    def addition(value1, value2):
        return (value1 + value2) / (1 + (value1 * value2) + EPSILON)

    def subtract(value1, value2):
        return (value1 - value2) / (1 - (value1 * value2) + EPSILON)

    def C(m, i):
        return math.factorial(m) / ((math.factorial(i) * math.factorial(m - i)) + EPSILON)

    def qx(i, x):
        if x == WIDTH - 1:
            return 0
        return C(m, i) * (np.power((x - x0) / (x1 - x), i) * np.power((x1 - x) / (x1 - x0), m))

    def qy(j, y):
        if y == HEIGHT - 1:
            return 0
        return C(n, j) * (np.power((y - y0) / (y1 - y), j) * np.power((y1 - y) / (y1 - y0), n))

    def p(i, j, x, y):
        return qx(i, x) * qy(j, y)

    def mapping(img, source, dest):
        return (dest[1] - dest[0]) * ((img - source[0]) / (source[1] - source[0])) + dest[0]

    def cal_ps_ws(m, n, w, h, gamma):
        ps = np.zeros((m, n, w, h))
        for i in range(m):
            for j in range(n):
                for k in range(w):
                    for l in range(h):
                        ps[i, j, k, l] = p(i, j, k, l)

        ws = np.zeros((m, n, w, h))
        for i in range(m):
            for j in range(n):
                ps_power_gamma = np.power(ps[i, j], gamma)
                for k in range(w):
                    for l in range(h):
                        ws[i, j, k, l] = ps_power_gamma[k, l] / (np.sum(ps[:, :, k, l]) + EPSILON)
        return ps, ws

    ps, ws = cal_ps_ws(m, n, WIDTH, HEIGHT, GAMMA)

    def cal_means_variances_lamdas(w, e_layer):
        means = np.zeros((m, n))
        variances = np.zeros((m, n))
        lamdas = np.zeros((m, n))

        def window_card(w):
            return np.sum(w)

        def window_mean(w, i, j):
            mean = 0
            for k in range(HEIGHT):
                for l in range(WIDTH):
                    mean = addition(mean, scalar_multiplication(w[i, j, l, k], e_layer[k, l]))
            return mean / window_card(w[i, j])

        def window_variance(w, i, j):
            variance = 0
            for k in range(HEIGHT):
                for l in range(WIDTH):
                    variance += w[i, j, l, k] * np.power(norm(subtract(e_layer[k, l], means[i, j])), 2)
            return variance / window_card(w[i, j])

        def window_lamda(w, i, j):
            return np.sqrt(IDEAL_VARIANCE) / (np.sqrt(variances[i, j]) + EPSILON)

        for i in range(m):
            for j in range(n):
                means[i, j] = window_mean(ws, i, j)
                variances[i, j] = window_variance(ws, i, j)
                lamdas[i, j] = window_lamda(ws, i, j)

        return means, variances, lamdas

    def window_enh(w, i, j, e_layer):
        return scalar_multiplication(lamdas[i, j], subtract(e_layer, means[i, j]))

    def image_enh(w, e_layer):
        new_image = np.zeros(e_layer.shape)
        width = e_layer.shape[1]
        height = e_layer.shape[0]
        for i in range(m):
            for j in range(n):
                win = window_enh(w, i, j, e_layer)
                w1 = w[i, j].T.copy()
                for k in range(width):
                    for l in range(height):
                        new_image[l, k] = addition(new_image[l, k], scalar_multiplication(w1[l, k], win[l, k]))
        return new_image

    def one_layer_enhancement(e_layer):
        new_E_image = image_enh(ws, e_layer)
        res_image = mapping(new_E_image, (-1, 1), (0, 255))
        res_image = np.round(res_image).astype(np.uint8)
        return res_image

    # Separação das camadas de cores
    layer_b, layer_g, layer_r = cv.split(img)

    # Mapeamento fuzzy para cada canal de cor
    e_layer_b = mapping(layer_b, (0, 255), (-1, 1))
    e_layer_g = mapping(layer_g, (0, 255), (-1, 1))
    e_layer_r = mapping(layer_r, (0, 255), (-1, 1))

    # Aplicar o processo de realce fuzzy para cada canal
    res_b = one_layer_enhancement(e_layer_b)
    res_g = one_layer_enhancement(e_layer_g)
    res_r = one_layer_enhancement(e_layer_r)

    # Combinar os canais aprimorados
    res_img_color = cv.merge([res_b, res_g, res_r])

    # Salvar a imagem colorida realçada
    plt.imsave('resultado_color.jpg', cv.cvtColor(res_img_color, cv.COLOR_BGR2RGB))
